<a href="https://colab.research.google.com/github/Armin-Abdollahi/Brain-Tumor-Diagnosis/blob/main/Brain_Tumor_Diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>